# **Grupo : Douglas Ferreira, Luiz Souza**.


In [1]:
import random
import ast
import string
import numpy as np
import matplotlib.pyplot as plt

In [2]:
prob_crossover = 0.9
prob_mutacao = 0.05
valores = [5,8,7,6,9,5,4,3]
pesos = [10,18,12,14,13,11,8,6]
tamanho_pop = 20
#valores = [5,8,7,6,9,5,4,3,5,7,9,8,3,5,4,6,7,9,3,7]
#pesos = [10,18,12,14,13,11,8,6,4,11,14,5,11,7,16,75,10,1,2,200]
qtde_itens = len(valores)
capacidade = 35



# Gera a população inicial

In [3]:
def gera_pop():
  populacao = []
  for i in range(tamanho_pop):
    individuo  = list(np.random.randint(2, size=qtde_itens))
    
    populacao.append(individuo)
  return populacao


# Função de aptidão:
Nessa função é empregado o conceito de penalização às soluções candidatas que ferirem a restrição de capacidade das mochilas. Caso a solução viole essa restrição o valor do fitness será penalizado para baixo. De forma mais clara e em termos matemáticos: 

$f'(\hat{x})=  \sum_{j=1}^{N} v_j x_j - Pen(\hat{x})$


sendo: $Pen(\hat{x}) = 
    \begin{cases}
      0, & \text{se}\  \hat{x} \ é \ válida \\
      \rho(\sum_{j=1}^{N} w_j x_j - cap), & \text{Caso contrário}
    \end{cases}
    
$ 

com 

$\rho = max(\frac{v_j}{w_j})$, $\forall_j =  \ {1,...,N}$

$v_j$ representa os valores de cada item candidato a entrar na mochila;

$w_j$ representa o peso que cada item candidato a entrar na mochila tem;

$x_j$ em uma solução com representação binária é 1 se o item entra na mochila e 0 se não entra

cap representa a capacidade total de uma determinada mochila.

In [4]:
def fitness(solution, valores:list=valores, pesos:list=pesos, capacidade:int=capacidade):
  beneficio = 0
  peso_total = 0
  lista_aux = []
  for j in range(len(pesos)):
    lista_aux.append(valores[j]/pesos[j])
  rho = max(lista_aux)

  for i in range(len(solution)):
    beneficio += ((solution[i]))*valores[i]
    peso_total += ((solution[i]))*pesos[i]
  if(peso_total > capacidade):
    return (beneficio - rho * (peso_total - capacidade));    
  else:
      return beneficio
      
    




# Função que calcula o benefício total e o peso total de uma determinada configuração de itens em uma mochila

In [5]:
def calcular_beneficio_e_peso(solucao):
  beneficio_total = 0
  peso_total = 0
  for i in range(len(solucao)):
    beneficio_total += valores[i]*solucao[i]
    peso_total += pesos[i]*solucao[i]
  return beneficio_total, peso_total


# Realiza crossover com um ponto de corte

In [6]:
def crossover(pais, prob_crossover):
  lista_aux = []
  if(random.random() > prob_crossover):
    return pais[0], pais[1]
  else:
    filho1 = []
    filho2 = []
    pontos_corte = int(np.random.randint(0, len(pais[0]), size=(1)))
    parte1_filho1 = pais[0][0:pontos_corte]
    parte1_filho2 = pais[1][0:pontos_corte]
    parte2_filho1 = pais[1][(-(len(pais[1]) - pontos_corte)):] 
    parte2_filho2 = pais[0][(-(len(pais[0]) - pontos_corte)):]
    filho1 = parte1_filho1 + parte2_filho1
    filho2 = parte1_filho2 + parte2_filho2
    return filho1, filho2

# Realiza a mutação Bit-Flip com uma probabilidade baixa(entre 0,02-0,1) 

In [7]:
def mutacao(individuo, prob_mutacao):
  if(random.random()>prob_mutacao):
    return individuo
  else:
    posicao_mut = np.random.randint(0,len(individuo))
    if(individuo[posicao_mut] == 0):
      individuo[posicao_mut] == 0
    else:
      individuo[posicao_mut] == 1
  return individuo

# Converte uma lista para um dicionário

In [8]:
def lista_to_dict(lista):
    it = iter(lista)
    dicionario = dict(zip(it, it))
    return dicionario

# Operador de Seleção Proporcional ao Fitness + Roleta;

In [9]:
def selecionapais(populacao):
    lista_pop_aptidoes = []
    aptidoes = []
    maior = 0
    for j in range(len(populacao)):      
      aptidoes.append(fitness(populacao[j], valores, pesos, capacidade))

    for k in range(len(populacao)):
      lista_pop_aptidoes.append(str(populacao[k]))
      lista_pop_aptidoes.append(aptidoes[k])
    pop = lista_to_dict(lista_pop_aptidoes)
    maximo = sum(pop.values())
    pick = random.uniform(0, maximo)
    atual = 0
    for individuo, valor_fitness in pop.items():
        atual += valor_fitness
        if atual > pick:         
            individuo= ast.literal_eval(individuo)#Converte string de uma lista para uma list de int
            return individuo

In [10]:
def evolucao():
  itens = []
  maior = 0
  populacao = gera_pop()
  for j in range(20000):
    for i in range(int(tamanho_pop/2)):#Todos os indivíduos de cada  geração são substituidos 
      pai1 = selecionapais(populacao)      
      pai2 = selecionapais(populacao)
      filho1, filho2 = crossover([pai1, pai2], prob_crossover)
      populacao[i] = filho1
      populacao[i+1] = filho2
  for k in range(len(populacao)):      
    if(fitness(populacao[k])>maior):
      maior = fitness(populacao[k])
      itens = populacao[k]
  return itens    




In [11]:
soluc = evolucao()

In [12]:
benef_tot, peso_tot = calcular_beneficio_e_peso(soluc)

if(peso_tot<=capacidade):
  print("Respeitou a restrição")
else:
  print("Não respeitou a restrição de capacidade, considere reexecutar a função evolução ")
print( "O benefício total foi de:", benef_tot," O peso total foi de ", peso_tot )

Respeitou a restrição
O benefício total foi de: 18  O peso total foi de  31


# *** Disclaimer /Conclusão***

# Pelo fato do algoritmo genético não ser uma classe de solucionadores exatos, em vez disso são meta-heurísticas, nem sempre a solução acima respeitará a  restrição de capacidade da mochila. O algoritmo é estocástico e a solução as vezes pode ser até ser factível, mas pode não ser a melhor. Por isso às vezes é mandatório que se calcule a solução várias vezes, que se faça um armazenamento delas e partir desse armazenamento é preciso determinar a solução, que ao mesmo tempo, mais otimize nossa função e  respeite a(s) restrição/restrições.